In [1]:
from sqlalchemy import create_engine, text

'''
An Engine is not a connection. It is a factory, that enables us to create connections to the DB as needed.
It also has connection pools to reuse connections to the same DB
'''
engine = create_engine("sqlite://", echo=True) #simply creates an in-memory DB for the duration of this Python program


In [ ]:
# The SQLAlchemy connection features an .execute() method that will run queries. To run a textual statement,
# create a statement from a string using Sqlalchemy's text() method and then execute it.
conn = engine.connect()
stmt = text("SELECT 'hello world' as greeting")
res = conn.execute(stmt) #Every execute returns a SqlAlchemy.engine.Result object, which is a proxy for a DBAPI cursor.
row = res.first() #this returns a Row. A Row object looks and acts like a Named Tuple
print(row)
print(row[0])
print(row.greeting)

('hello world',)
hello world
hello world


In [ ]:
res = conn.execute(stmt) #.first() closes the earlier connection as it interprets it as we have gotten our data.


#To get lots of rows back, we have iteration
for row in res:
    print(f"{row=}")

res = conn.execute(stmt) #.first() closes the earlier connection as it interprets it as we have gotten our data.

#To get all rows as a list
list_of_results = res.all()

#To iterate through first column of each row
for greeting in res.scalars():
    print(f"{greeting=}") #each 'greeting' is just a native type (here string)

row=('hello world',)
greeting='hello world'


In [2]:
'''
We should favour using context managers for the connections.
There are two variations in committing transactions in SQLAlchemy 2.0
1) Commit as you go: as SQL is run, a transaction begins, which then has to be committed as you go along
'''
with engine.connect() as connection:
    connection.execute(
        text("CREATE TABLE employee (emp_id PRIMARY KEY, emp_name VARCHAR, fullname VARCHAR)")
    )

    insrt_stmt = text("INSERT INTO employee (emp_name, fullname) VALUES (:name, :fullname)")
    connection.execute(
        insrt_stmt, {"name": "Spongebob", "fullname": "Spongebob Squarepants"}
    )

    connection.commit()

2026-02-24 00:15:32,073 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-24 00:15:32,074 INFO sqlalchemy.engine.Engine CREATE TABLE employee (emp_id PRIMARY KEY, emp_name VARCHAR, fullname VARCHAR)
2026-02-24 00:15:32,074 INFO sqlalchemy.engine.Engine [generated in 0.00181s] ()
2026-02-24 00:15:32,077 INFO sqlalchemy.engine.Engine INSERT INTO employee (emp_name, fullname) VALUES (?, ?)
2026-02-24 00:15:32,078 INFO sqlalchemy.engine.Engine [generated in 0.00128s] ('Spongebob', 'Spongebob Squarepants')
2026-02-24 00:15:32,079 INFO sqlalchemy.engine.Engine COMMIT


In [2]:
'''
2) Begin Once: the transaction begins as an explicit block that commits when complete
'''
with engine.begin() as connection:
        connection.execute(
        text("CREATE TABLE employee (emp_id PRIMARY KEY, emp_name VARCHAR, fullname VARCHAR)")
    )

        insrt_stmt = text("INSERT INTO employee (emp_name, fullname) VALUES (:name, :fullname)")
        connection.execute(
            insrt_stmt, {"name": "Spongebob", "fullname": "Spongebob Squarepants"}
    )

2026-02-24 00:19:08,956 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-24 00:19:08,958 INFO sqlalchemy.engine.Engine CREATE TABLE employee (emp_id PRIMARY KEY, emp_name VARCHAR, fullname VARCHAR)
2026-02-24 00:19:08,958 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ()
2026-02-24 00:19:08,960 INFO sqlalchemy.engine.Engine INSERT INTO employee (emp_name, fullname) VALUES (?, ?)
2026-02-24 00:19:08,960 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ('Spongebob', 'Spongebob Squarepants')
2026-02-24 00:19:08,962 INFO sqlalchemy.engine.Engine COMMIT
